In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy.sql import text

import baikalfunctions as bfunc
import scheme_mar2023 as scheme
import mytools

import glob

In [ ]:
# pd.set_option('display.min_rows', 100)
%matplotlib inline
plt.style.use('dark_background')   # try another styles: 'classic'
plt.rcParams['figure.figsize'] = [15, 3]
#plt.subplots_adjust(top=1, left=0, right=1, bottom=0)

In [ ]:
path=r'G:\1_Data1\!AMK93\Jury'
files=glob.glob(path+'\*\*\*.dat')
len(files)

In [ ]:
df_meteo = pd.DataFrame()
for f in files:
    dft = pd.read_csv(f,
                     encoding_errors='ignore',
                     header=None,
                     skiprows=0,
                     sep='\s+',
                      na_values=['-'],
                     names=['date',
                            'time',
                            'utc_cor',
                            'c13_picarro',
                            'ch4_picarro',
                            'co2_picarro',
                            'h2o_picarro?',
                            'latitude', 
                            'longitude', 
                            'Altitude', 'RVCourse', 'RVCourseMgps', 'RVSpeed', 'x1', 'RVCourseMcom', 'Pitch', 'Roll',
                            'x9', 'x10',
                            'Press',
                            'Tair',
                            'RH',
                            'WindSpeed', 'WindDir', 'WinDirM', 'WindSpeedRel', 'WindDirRel', 'x16', 'x17'],
                     parse_dates=[['date', 'time']],
                     # dtype={'ch4_picarro': np.float64} 
                      
                    )
    df_meteo = pd.concat([df_meteo, dft], ignore_index=True)
    print(f)
df_meteo

In [ ]:
df_meteo.rename(columns=scheme.NAME_CONV_BAS, inplace=True)
df_meteo.sort_values(by='DateTime', inplace=True)
df_meteo.reset_index(inplace=True, drop=True)

In [ ]:
df_meteo.info()

In [ ]:
columns_to_stay = ['DateTime', 'TempAir', 'Press', 'RH', 'Latitude', 'Longitude', 'RVCourse', 'RVCourseMgps', 'RVSpeed', 'Pitch', 'Roll', 'WindSpeed', 'WindDir']
columns_to_drop = [x for x in df_meteo.columns if x not in columns_to_stay]
print(columns_to_drop)
df_meteo.drop(columns_to_drop, axis='columns', inplace=True, errors='ignore')
df_meteo


In [ ]:
df_meteo['DateTime'] = df_meteo['DateTime'] + pd.Timedelta(hours=3)
df_meteo

In [ ]:
datetime_start = pd.to_datetime('2023-11-08 00:00')
datetime_stop = pd.to_datetime('2023-12-07 00:00')
df_meteo = df_meteo.loc[(df_meteo['DateTime'] > datetime_start) & (df_meteo['DateTime'] < datetime_stop)]

In [ ]:
plt.plot(df_meteo['DateTime'], df_meteo['TempAir'], 'r-')

In [ ]:
plt.plot(df_meteo['Longitude'], df_meteo['Latitude'], 'g.')
plt.plot(35, 90, 'b.')

In [ ]:
df_picarro = pd.read_csv(r'G:\1_Data1\!AMK93\picarro\picarroCor.dat',
                 sep='\t',
                 usecols=[1,2,4],
                 parse_dates=['DateTime'],
                 dayfirst=False,
                )
df_picarro.info()

In [ ]:
df_picarro = df_picarro.resample('20S', on='DateTime').mean()

In [ ]:
df = pd.merge(left=df_picarro, right=df_meteo, how='left', left_on='DateTime', right_on='DateTime', suffixes=['_p', '_m'])

In [ ]:
df.sort_values(by='DateTime')
df

In [ ]:
cols = ['TempAir', 'Press', 'RH', 'Latitude', 'Longitude', 'RVCourse', 'RVCourseMgps', 'RVSpeed', 'Pitch', 'Roll', 'WindSpeed', 'WindDir']
for col in cols:
    df[col].interpolate(method='linear', axis='rows', inplace=True)


In [ ]:
df

In [ ]:
df.loc[55000:60000, ['Longitude', 'Latitude']].plot(x='Longitude', y='Latitude')

In [ ]:
df.to_csv(r'G:\1_Data1\!AMK93\amk93_all.dat',
          sep='\t',
         )

In [ ]:
plt.rcParams['figure.figsize'] = [10, 12]
fig, ax = plt.subplots()
#ax.set_xlim(106.2, 109.4)
#ax.set_ylim(52, 55)

ax.plot(obris['long'], obris['lat'], 'b-')
ax.scatter(x='long', y='lat', c=cCH4wtrlog_all, s=50, marker='o', linewidth=0, cmap='rainbow', alpha=0.02, data=allMethane_df, )
ax.scatter(df['Longitude'], df['Latitude'], c=df['cCH4wtrlog'], s=30, marker='o', linewidth=0, cmap='rainbow', alpha=0.4,)
fig.savefig(saveImgPath+'CH4wtrSpatialJun2023.png', transparent=True)

In [ ]:
df

In [ ]:
dft = pd.read_clipboard(header=None, names=['DateTime', 'Stations'], na_values='--', parse_dates=['DateTime'], dayfirst=True)
dft.info()

In [ ]:
df = df.merge(dft, left_on='DateTime', right_on='DateTime', how='left')

In [ ]:
dft['Stations'].fillna(method='ffill', inplace=True)
dft

In [ ]:
dft

In [ ]:
df = pd.merge(left=df, right=dft, how='left', left_on='DateTime', right_on='DateTime')

In [ ]:
df['Stations'] = df['Stations'].str.replace('_', '.')
df.loc[(df['DateTime']==pd.to_datetime('2023-11-19 11:55:00'))]

In [ ]:
df = df.astype({'Stations': np.float32})
df.dtypes

In [ ]:
df.loc[(df.Stations)==7733.1]


In [ ]:
df

In [ ]:
import physic
def get_speed_column(df):
    torad = 2 * np.pi / 360
    dt_sec = np.int16(df['DateTime'].astype(np.int64)/10**9 - df['DateTime'].shift(1).astype(np.int64)/10**9)
    lat_rad = df['Latitude'] * torad
    long_rad = df['Longitude'] * torad
    dsigma = np.arccos(np.sin(lat_rad)*np.sin(lat_rad.shift(1)) + np.cos(lat_rad)*np.cos(lat_rad.shift(1))*np.cos(long_rad.shift(1)-long_rad))
    df['mVSpeed'] = physic.R_EARTH * dsigma * 1000 / dt_sec
get_speed_column(df)

In [ ]:
df.loc[80000:90000].plot('DateTime', 'mVSpeed')

In [ ]:
get_speed_column(df)
df

In [ ]:
df.to_clipboard()

In [ ]:
dff = pd.read_clipboard(header=None, names=['DateTime', 'cCH4', 'cCO2', 'FluxCH4', 'FluxCO2'], parse_dates=['DateTime'], dayfirst=True, na_values=np.nan)
dff.info()

In [ ]:
dff

In [ ]:
dff = df.merge(dff, left_on='DateTime', right_on='DateTime', how='left')

In [ ]:
df['x'].describe()

In [ ]:
dff.loc[:,['cCH4', 'cCO2', 'FluxCH4', 'FluxCO2']].to_clipboard()